# Yahoo New Crawl hint

This notebook demostrate how to crawl data from Yahoo News (https://tw.news.yahoo.com/).

However, it's incomplete. You'll have to use some advanced techniques to collect enough data for document classification.

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
req = requests.get('https://tw.news.yahoo.com/')

In [3]:
soup = BeautifulSoup(req.text, 'lxml')

Get categories of Yahoo News

In [4]:
categories = []
for category in soup.select(".nr-applet-nav-item.nr-list-link.openSubNav"):
    categories.append(category.attrs['href'].strip('/').split('/')[-1])
    print(category.attrs['href'])

https://tw.news.yahoo.com
https://tw.news.yahoo.com/politics
https://tw.news.yahoo.com/finance
https://tw.news.yahoo.com/entertainment
https://tw.news.yahoo.com/sports
https://tw.news.yahoo.com/society
https://tw.news.yahoo.com/local
https://tw.news.yahoo.com/world
https://tw.news.yahoo.com/lifestyle
https://tw.news.yahoo.com/health
https://tw.news.yahoo.com/technology
https://tw.news.yahoo.com/travel
https://tw.news.yahoo.com/weather/
https://tw.news.yahoo.com/odd
https://tw.news.yahoo.com/blogs
https://tw.news.yahoo.com/video
https://tw.news.yahoo.com/topic/520survey
https://tw.news.yahoo.com/topic/French_Election
https://tw.news.yahoo.com/topic/SouthKorea_Election
https://tw.news.yahoo.com/topic/pension
https://tw.news.yahoo.com/topic/Laborlaw
https://tw.news.yahoo.com/topic/LIN-YIHAN


remove unwanted categories

In [6]:
categories = categories[1: -6]
categories

['politics',
 'finance',
 'entertainment',
 'sports',
 'society',
 'local',
 'world',
 'lifestyle',
 'health',
 'technology',
 'travel',
 'weather',
 'odd',
 'blogs',
 'video']

### Get news list

In [7]:
req = requests.get('https://tw.news.yahoo.com/{}'.format(categories[0]))
soup = BeautifulSoup(req.text, 'lxml')

In [10]:
from urllib.parse import urljoin
urls = set()
for article in soup.select(".js-stream-content a"):
    url = urljoin('https://tw.news.yahoo.com/', article.attrs['href'])
    urls.add(url)
print(urls)

{'https://tw.news.yahoo.com/%E5%8E%9F%E4%BD%8F%E6%B0%91%E5%8A%83%E8%A8%AD%E5%82%B3%E7%B5%B1%E9%A0%98%E5%9F%9F%E4%B8%8D%E6%98%AF%E5%9C%A8%E6%90%B6%E5%9C%B0-%E6%BC%A2%E4%BA%BA%E5%88%A5%E6%93%94%E5%BF%83-131659896.html', 'https://tw.news.yahoo.com/%E5%A5%B3%E7%AB%A5%E5%89%B2%E5%96%89%E6%A1%88%E4%BA%8C%E5%AF%A9-%E9%BE%94%E9%87%8D%E5%AE%89%E7%B6%AD%E6%8C%81%E7%84%A1%E6%9C%9F%E5%BE%92%E5%88%91-032034311.html', 'https://tw.news.yahoo.com/wha-115909431.html', 'https://tw.news.yahoo.com/%E8%AD%B0%E5%93%A1%E8%A6%81%E8%B3%87%E6%96%99%E4%B8%8D%E7%B5%A6-%E5%8F%B0%E5%8D%97%E5%B8%82%E5%BA%9C-%E6%B6%89%E5%80%8B%E8%B3%87-%E5%9C%96-115100899.html', 'https://tw.news.yahoo.com/%E8%94%A1%E7%95%B6%E5%B1%80%E7%B7%A3%E4%BD%95%E6%94%B6%E4%B8%8D%E5%88%B0%E4%B8%96%E8%A1%9B%E7%9A%84%E9%82%80%E8%AB%8B%E4%BF%A1%EF%BC%9F-111717348.html', 'https://tw.news.yahoo.com/%E6%9C%AA%E7%8D%B2wha%E9%82%80%E8%AB%8B%E5%87%BD-%E5%BA%9C-%E9%81%BA%E6%86%BE%E8%88%87%E4%B8%8D%E6%BB%BF-134300425.html', 'https://tw.news.yahoo.com/26-13

#### Pack it into a function

In [22]:
def get_news_links(category):
    page_url = 'https://tw.news.yahoo.com/{}'.format(category)
    req = requests.get(page_url)
    soup = BeautifulSoup(req.text, 'lxml')

    return {urljoin(page_url, article.attrs['href']) for article in soup.select(".js-stream-content a")}

### Get links from every category

In [24]:
links = {}
for category in categories:
    links[category] = get_news_links(category)

In [26]:
links.keys()

dict_keys(['politics', 'finance', 'entertainment', 'sports', 'society', 'local', 'world', 'lifestyle', 'health', 'technology', 'travel', 'weather', 'odd'])

## Get news content

get the html document

In [29]:
req = requests.get('https://tw.news.yahoo.com/%E6%94%BF%E9%99%A2-%E6%8E%A8%E5%8B%95%E5%B9%B4%E6%94%B9%E7%B5%95%E4%B8%8D%E5%8B%95%E6%90%96-160000837.html')
soup = BeautifulSoup(req.text, 'lxml')

In [33]:
soup.select_one('header.canvas-header h1').text

'政院：推動年改絕不動搖'

In [36]:
soup.select_one('div.auth-prov-soc').text

'【記者潘鴻志、王志誠／台北報導】'

In [40]:
for p in soup.select('div.canvas-body p'):
    print(p.text)

行政院副院長林錫耀於昨（六）日代表林全院長出席「中華民國工商協進會中部工商午餐會」時表示，去（一○五）年五二○新政府上台後，積極解決各項問題，例如台灣面臨少子女化、高齡化等危機，因此，行政團隊積極推動年金改革，決心不會動搖，年金改革最重要的法案，已在立法院的委員會通過，盼在一個月內能完成二、三讀，也希望其它相關法案都能順利通過。
林副院長指出：除面對問題，推動改革外，政府也極力進行各項建設，林全院長今天就到台南訪視沙崙綠能科學城，總統也非常重視產業界的意見，充分聽取產業界的心聲，希望有效且積極解決問題，目前提出的八年八八○○億「前瞻基礎建設計畫」就是為了加速推動國家未來所需的建設，帶動經濟成長的動能，但「前瞻基礎建設計畫特別條例」目前還無法在立法院的委員會通過，盼各界給予支持，讓有利經濟民生的計劃儘速啟動。
林副院長並強調：新政府有決心，有計畫，堅定務實推動各項改革，並加速進行建設。台灣過去很長一段時間，經濟並不景氣。但新政府上台後，最近公布的一些經濟數據可發現，目前景氣已在穩定復甦中，政府希望在此時期，一方面加速各項基礎建設，擴大財政支出，帶動經濟成長；另外，也逐步推動各項經濟轉型政策，包括：「五＋二」創新產業計畫，期能透過產業創新帶動經濟轉型，讓台灣有更多元的經濟發展動能。
林副院長也指出：影響企業投資的障礙，例如水、電、人才、土地及環評等，政府都瞭解，也會積極溝通說明，希望取得更好的共識。


Pack it into a function

In [6]:
def get_new_content(url):
    content = []
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    content.append(soup.select_one('header.canvas-header h1').text)
    content.append(soup.select_one('div.auth-prov-soc').text)
    for p in soup.select('div.canvas-body p'):
        content.append(p.text)
    return '\n'.join(content)

In [11]:
url = 'https://tw.news.yahoo.com/-043018999.html'
print(get_new_content(url))

八田與一銅像修復揭幕 長孫盼傳承到後世
政治中心／綜合報導民報2017年5月7日
台南烏山頭水庫八田與一銅像遭斷頭，日前完成修復，今（7）日上午舉行揭幕儀式。從日本趕來出席的八田與一長孫八田修一說，不敢相信這麼快就修復了，感到很開心，但他不是為銅像來的，而是來確認台日間的友誼沒變，而秀麗的風景沒變，景色依舊，他希望一切能傳承到後世。
台南市長賴清德則表示，八田與一銅像遭人蓄意破壞，市府未能妥善保護，對於家屬表達遺憾，但此種破壞行為不會影響台日感情，台日感情甚至會因為歷經考驗而更好。
八田與一銅像4月15日遭前台北市議員李承龍、邱姓女子斷首後，備受關注，連日本媒體也大幅報導；在頭部遍尋不著的情形下，王姓專業技師以奇美博物館典藏的胸像，接上銅像，完成修復。明天八田與一忌日，追思活動將如期舉行。
上午揭幕式包括台南市長賴清德、日本金澤市長山野之義、加賀市副市長河合篤史、日本台灣交流協會高雄事務所長中郡錦藏、日本金澤市訪問團，八田修一夫婦等家屬等多人參加，共同見證八田與一銅像的修復，並聲言台日友誼穩固。
金澤市長山野之義表示，這銅像是當初台灣朋友幫忙建造，台灣的朋友懷著感恩之心，幫我們故鄉的大前輩建造這銅像；感到開心的同時，身為台南市的友人他感到非常驕傲。
他表示，當然（銅像斷頭）這是一個令人不悅且傷心的事件，但台南與金澤、日本與台灣的友誼更深厚強固，他回到日本後，會把這好消息傳達給市民。
八田修一則說，看到祖父銅像在這麼短時間內修復，即使現場親眼目睹，他還是不敢相信；但他不是為銅像來的，而是要確認台日間的友誼沒變，不會因銅像而損及台日友誼。
八田修一表示，他的祖父75年前坐在此地，但這裡都沒有改變，水庫一樣在出水，風景依然秀麗，他希望能一直傳承到後世。
賴清德則表示，金澤市的傑出市民八田在八、九十年前東京帝大畢業後就來到台南，在物質相當缺乏的年代，建造了一個當時亞洲最現代化的水庫，不僅嘉慧農民，也促成了經濟發展。許許多多的農民、台灣的朋友，日本的好朋友們，大家都飲水思源，感念八田與一先生的偉大貢獻，集資興建銅像。
賴清德也表示，八田銅像4月26日就已修復完成，很有效率，沒有留下傷害的痕跡，非常感謝！這起令人遺憾的事件，目標是要破壞日本跟台灣的感情，慶幸的是台灣社會反對這樣的行為。他說，感情要經過考驗，考驗過的感情才是真感情，台日感情不僅沒變，還會更好。 
嘉南農田水利會會

news id parse function

In [11]:
import re
url_re = re.compile('-(\d{7,10})')

def parse_nid(url):
    m = url_re.search(url)
    return m.group(1)

In [63]:
parse_nid(url)

'125910139'

In [14]:
import os
for c in categories:
    os.mkdir(c)

In [261]:
for category, urls in links.items():
    for url in urls:
        print(url)
        nid = parse_nid(url)
        filename = '{0}/{1}'.format(category, nid)
        
        if os.path.isfile(filename):
            continue

        content = get_new_content(url)
        with open(filename, 'w') as f:
            print(content, file=f)

https://tw.news.yahoo.com/-105909176.html
https://tw.news.yahoo.com/-075910832.html
https://tw.news.yahoo.com/%E8%BF%8E%E5%A8%B6%E6%96%B0%E5%A8%98%E5%97%A8%E9%81%8E%E9%A0%AD-%E6%8E%A8%E6%96%B7%E6%AC%84%E6%9D%867%E4%BA%BA%E5%A2%9C%E6%A8%93-054600419.html
https://tw.news.yahoo.com/%E8%98%87%E8%8A%B1%E6%94%B9%E4%B8%8D%E6%8E%A5%E5%9C%8B%E4%BA%94-%E5%AE%9C%E8%98%AD%E7%B8%A3%E9%95%B7%E6%80%A8%E8%98%87%E6%BE%B3%E5%A1%9E%E7%88%86-003200571.html
https://tw.news.yahoo.com/%E5%91%82%E7%A7%80%E8%93%AE%E6%8E%A8%E5%92%8C%E5%B9%B3%E4%B8%AD%E7%AB%8B%E5%85%AC%E6%8A%95-%E5%9C%96-135101289.html
https://tw.news.yahoo.com/%E8%AD%B0%E5%93%A1%E5%86%8D%E7%88%86%E6%9C%89%E7%8B%BC%E5%B8%AB-%E9%99%B3%E8%8F%8A%E6%80%92-%E5%88%A5%E5%83%8F%E9%80%A3%E7%BA%8C%E5%8A%87-111900642.html
https://tw.news.yahoo.com/%E7%8D%A8%E5%AE%B6-%E6%B4%AA%E7%A7%80%E6%9F%B1%E6%85%8B%E5%BA%A6%E5%87%BA%E7%8F%BE%E8%BD%89%E5%9C%9C-10%E6%97%A5%E8%BE%AF%E8%AB%96%E6%9C%83%E6%9C%89%E6%9C%9B%E5%87%BA%E5%B8%AD-%E6%88%B0-042400430.html
https://tw.

https://tw.news.yahoo.com/%E6%B4%AA%E7%A8%B1-%E8%B7%AF%E7%9B%B8%E6%8C%BA-%E8%A8%B1%E6%AD%B7%E8%BE%B2-%E4%BB%A5%E5%89%8D%E6%8C%BA%E6%B4%AA-%E7%8F%BE%E5%9C%A8%E6%8C%BA%E9%83%9D-062033317.html
https://tw.news.yahoo.com/-072909340.html
https://tw.news.yahoo.com/%E5%8F%B0%E7%81%A3%E6%98%AF%E6%88%91%E6%9C%AA%E4%BE%86%E5%9F%8B%E9%AA%A8%E8%99%95-%E6%B4%AA%E7%A7%80%E6%9F%B1-%E6%88%91%E6%98%AF%E5%8F%B0%E7%81%A3%E4%BA%BA-%E4%B9%9F%E6%98%AF%E4%B8%AD%E5%9C%8B%E4%BA%BA-100000050.html
https://tw.news.yahoo.com/%E6%8E%A8%E5%BB%A3%E5%95%86%E5%9C%88-%E6%9F%AF%E6%96%87%E5%93%B2%E9%A8%8E%E9%87%8D%E6%A9%9F%E5%AE%A3%E5%82%B3-115400274.html
https://tw.news.yahoo.com/%E8%88%AA%E7%89%B9%E9%83%A8%E7%89%B9%E8%A8%93%E4%B8%AD%E5%BF%83%E6%AD%A5%E6%A7%8D%E7%AB%9F%E4%B8%9F%E4%BA%86-%E9%80%9A%E5%A0%B1%E6%86%B2%E5%85%B5%E5%81%B5%E6%9F%A5-072100781.html
https://tw.news.yahoo.com/%E5%BE%9E%E5%8F%B0%E5%A4%A7%E9%80%80%E4%BC%91%E9%A0%9818-%E6%9E%97%E8%90%AC%E5%84%84-%E7%85%A7%E8%A6%8F%E5%AE%9A-111900064.html
https://tw.news

https://tw.news.yahoo.com/%E5%8F%B0%E5%B9%A3%E5%BC%B7%E5%8D%87%E5%87%BA%E5%8F%A3%E4%BA%AE%E9%BA%97%E5%AD%98%E9%9A%B1%E6%86%82-160000815.html
https://tw.news.yahoo.com/%E5%A6%82%E6%84%8F%E8%8F%9C-215007321--finance.html
https://tw.news.yahoo.com/%E9%A0%AD-%E9%81%AD-%E6%B8%AF%E4%BA%A4%E6%89%80%E6%8E%A8%E9%9B%99%E5%B9%A3%E8%A8%88%E5%83%B9%E9%BB%83%E9%87%91%E5%AF%A6%E7%89%A9%E6%9C%9F%E8%B2%A8-061119269.html
https://tw.news.yahoo.com/%E6%B9%9B%E9%8C%A6%E6%BA%90%E7%8D%B2%E9%81%B8%E6%95%B4%E5%85%A8%E6%80%A7%E5%81%A5%E5%BA%B7%E7%AE%A1%E7%90%86%E5%8D%94%E6%9C%83%E7%90%86%E4%BA%8B%E9%95%B7-%E5%9C%96-042701167--finance.html
https://tw.news.yahoo.com/%E9%81%A0%E8%88%AA%E9%A1%98%E6%92%A4%E9%8A%B7%E8%B3%87%E9%81%A3%E6%B1%BA%E5%AE%9A-%E6%AD%A1%E8%BF%8E17%E7%A9%BA%E6%9C%8D%E5%93%A1%E5%9B%9E%E4%BB%BB-120528479--finance.html
https://tw.news.yahoo.com/%E6%9E%97%E8%90%AC%E5%84%84%E8%BE%A6%E9%80%80%E4%BC%91-%E6%9D%8E%E4%BE%86%E5%B8%8C-%E6%86%91%E4%BB%80%E9%BA%BC-130129950.html
https://tw.news.yahoo.com/ube

https://tw.news.yahoo.com/%E5%B0%8F%E7%94%9C%E7%94%9C%E5%B8%83%E8%98%AD%E5%A6%AE-6%E6%9C%88%E9%A6%96%E5%BA%A6%E4%BE%86%E5%8F%B0%E9%96%8B%E5%94%B1-%E5%9C%96-073500778.html
https://tw.news.yahoo.com/iu-%E7%9A%84%E5%8A%9B%E9%87%8F-palette-%E7%99%BB%E5%91%8A%E7%A4%BA%E7%89%8C-world-albums-%E6%A6%9C%E5%86%A0%E8%BB%8D%E5%AF%B6%E5%BA%A7-034200218.html
https://tw.news.yahoo.com/%E8%BF%8E%E4%B8%96%E5%A4%A7%E9%81%8B-%E5%9C%8B%E9%9A%9B%E6%97%A55%E6%9C%88%E6%8E%A5%E5%8A%9B%E6%9A%96%E5%A0%B4-%E4%B8%BB%E9%A1%8C%E6%A9%AB%E8%B7%A85%E6%B4%B2-%E6%8E%A8%E5%87%BA230%E5%80%8B%E5%B8%82%E9%9B%86-100115640.html
https://tw.news.yahoo.com/%E5%8F%B0%E8%A3%94%E5%A5%B3%E6%98%9F%E5%97%86%E8%BE%A3%E6%95%A2%E8%A8%80-%E5%BE%81%E6%9C%8D%E5%A5%BD%E8%90%8A%E5%A1%A2-160000027.html
https://tw.news.yahoo.com/%E5%B7%B4%E9%BB%8E%E9%90%B5%E5%A1%94%E6%8E%9B%E6%A8%99%E8%AA%9E-%E7%B6%A0%E8%89%B2%E5%92%8C%E5%B9%B3%E4%B8%8D%E6%8C%BA%E7%91%AA%E7%90%B3%E9%9B%B7%E6%9C%8B-083654627.html
https://tw.news.yahoo.com/%E9%80%9A%E9%9D%88%E5%B

https://tw.news.yahoo.com/%E5%91%A8%E6%8F%9A%E9%9D%92%E6%9B%AC%E8%BE%A3%E7%85%A7-%E7%A9%BF%E7%B7%8A%E8%BA%AB%E8%A1%A3%E6%B8%BE%E5%9C%93%E7%BE%8E%E8%83%B8%E5%BD%88%E5%87%BA-011800924.html
https://tw.news.yahoo.com/%E9%AB%98%E5%85%AC%E5%B1%80%E7%94%9F%E6%85%8B%E7%B4%80%E9%8C%84%E7%89%87-%E7%8D%B2%E7%BE%8E%E5%9C%8B%E5%BD%B1%E5%B1%95%E7%99%BD%E9%87%91%E7%8D%8E-131659054.html
https://tw.news.yahoo.com/vixx%E6%96%B0%E5%B0%88%E8%BC%AF%E7%9A%84%E9%80%A0%E5%9E%8B-kpop%E4%B9%9F%E7%8E%A9%E7%A9%BF%E8%B6%8A%E4%BA%86-021400131.html
https://tw.news.yahoo.com/%E5%8E%9F%E8%AB%92%E8%80%81%E5%85%AC%E5%81%B7%E5%90%83%E5%B0%8F%E4%B8%89-%E5%A5%B3%E6%98%9F%E9%A0%86%E5%88%A9%E7%94%A2%E4%B8%8B%E7%94%B7%E5%AC%B0-072600004.html
https://tw.news.yahoo.com/%E9%A6%AC%E5%B0%BC%E6%8B%89%E4%BD%9B%E5%85%89%E5%B1%B1-flip%E9%9F%B3%E6%A8%82%E6%9C%83%E6%BC%94%E5%87%BA%E6%88%90%E5%8A%9F-014441731.html
https://tw.news.yahoo.com/md-photo-%E5%85%A8%E5%AD%9D%E7%9B%9B%E6%9D%8E%E5%9F%BA%E5%AE%87%E7%AD%89%E5%87%BA%E5%B8%AD%E5%90%B3

...

https://tw.news.yahoo.com/%E9%99%B8%E6%97%A5%E9%87%91%E8%9E%8D%E5%B0%8D%E8%A9%B1-%E5%95%86%E4%BF%9D%E8%AD%B7%E4%B8%BB%E7%BE%A9%E5%92%8C%E5%8C%97%E9%9F%93%E7%AD%89%E9%A2%A8%E9%9A%AA-060219777.html
https://tw.news.yahoo.com/%E5%A7%94%E5%85%A7%E7%91%9E%E6%8B%89%E5%8B%95%E4%BA%82%E5%A2%9E%E8%87%B3%E9%80%BE36%E6%AD%BB-%E6%90%B6%E5%8A%AB%E5%9B%9B%E8%B5%B7-030115166.html
https://tw.news.yahoo.com/%E6%B3%95%E5%9C%8B%E6%96%B0%E8%A6%8F%E4%B8%8A%E8%B7%AF-%E8%B7%9F%E7%B4%99%E7%89%87%E4%BA%BA%E6%A8%A1%E7%89%B9%E5%85%92%E8%AA%AA%E6%8E%B0-155002264.html
https://tw.news.yahoo.com/%E4%BA%BA%E9%81%93%E8%80%83%E9%87%8F-%E5%9F%83%E5%8F%8A%E9%87%8D%E9%96%8B%E6%8B%89%E6%B3%95%E7%A9%BF%E8%B6%8A%E7%AB%993%E5%A4%A9-143502824.html
https://tw.news.yahoo.com/%E5%86%92%E5%85%85%E8%AD%A6%E5%AF%9F%E8%A1%8C%E9%A8%99%E7%99%BE%E8%90%AC-%E5%8F%B0%E7%94%B7%E6%96%B0%E5%8A%A0%E5%9D%A1%E8%A2%AB%E9%80%AE-133729724.html
https://tw.news.yahoo.com/%E9%99%B8%E5%BC%95%E9%80%B2%E8%98%87%E6%84%B735-%E5%88%86%E6%9E%90-%E6%AA%A2%E9%A

https://tw.news.yahoo.com/%E6%AD%A1%E6%85%B6%E4%BA%94%E4%BA%94-%E5%B1%8F%E6%9D%B1%E8%88%9E%E5%A4%8F%E8%97%9D%E8%A1%93%E7%AF%80%E7%99%BB%E5%A0%B4-070100129.html
https://tw.news.yahoo.com/%E8%A1%97%E9%A0%AD%E9%81%87%E9%B4%9B%E9%B4%A6%E5%A4%A7%E7%9B%9C-9%E6%97%AC%E7%BF%81%E6%BF%80%E5%8B%95%E5%BE%9E%E8%BC%AA%E6%A4%85%E7%AB%99%E8%B5%B7-062033677.html
https://tw.news.yahoo.com/%E5%A4%96%E5%9F%94%E5%8D%80%E5%90%84%E7%95%8C%E6%AD%A1%E6%85%B6%E6%AF%8D%E8%A6%AA%E7%AF%80-%E8%A1%A8%E6%8F%9A%E6%A8%A1%E7%AF%84%E6%AF%8D%E8%A6%AA%E8%88%87%E5%AD%9D%E5%AD%90-075526524.html
https://tw.news.yahoo.com/%E7%89%B9%E6%AE%8A%E6%95%99%E8%82%B2%E5%AD%B8%E7%94%9F%E6%8C%91%E6%88%B0%E6%97%A5-%E6%9C%B1-%E6%8C%91%E6%88%B0%E8%87%AA%E6%88%91-%E8%BF%8E%E5%90%91%E6%9C%AA%E4%BE%86-095841664.html
https://tw.news.yahoo.com/%E4%BB%8A%E5%BD%A9539%E9%A0%AD%E7%8D%8E-2%E6%B3%A8%E4%B8%AD%E7%8D%8E-140630261.html
https://tw.news.yahoo.com/%E9%83%AD%E5%A9%9E%E6%B7%B3%E5%86%8D%E7%A0%B4%E7%B4%80%E9%8C%84-%E6%94%BE%E7%9C%BC%E4%B8%96%E5%

https://tw.news.yahoo.com/%E8%BF%8E%E4%B8%96%E5%A4%A7%E9%81%8B-%E5%9C%8B%E9%9A%9B%E6%97%A55%E6%9C%88%E6%8E%A5%E5%8A%9B%E6%9A%96%E5%A0%B4-%E4%B8%BB%E9%A1%8C%E6%A9%AB%E8%B7%A85%E6%B4%B2-230%E5%80%8B%E5%B8%82%E9%9B%86-081100144.html
https://tw.news.yahoo.com/%E5%8F%B0%E8%83%9E%E8%AD%89%E8%87%AA%E5%8A%A9%E5%A0%B1%E5%88%B0-%E9%99%B8%E6%A9%9F%E5%A0%B4%E5%8D%87%E7%B4%9A%E5%8D%83%E5%8F%B0%E6%A9%9F%E5%99%A8-121328495.html
https://tw.news.yahoo.com/-122909443.html
https://tw.news.yahoo.com/%E6%9C%8D%E5%8B%99%E8%B2%BB%E5%85%A8%E5%85%A5%E8%80%81%E9%97%86%E5%8F%A3%E8%A2%8B-%E6%97%A5%E6%9C%AC%E4%BA%BA%E9%BB%9E%E5%87%BA%E7%9B%B2%E9%BB%9E-100500195.html
https://tw.news.yahoo.com/%E5%A4%A7%E5%AD%B8%E7%94%9F%E7%A7%9F%E5%B1%8B-%E4%BA%A4%E5%B1%8B%E5%89%8D%E6%87%89%E8%A9%B2%E7%9F%A5%E9%81%93%E7%9A%845%E4%BB%B6%E4%BA%8B-042700257.html
https://tw.news.yahoo.com/%E4%B8%8D%E5%BF%8D%E5%AD%A9%E5%AD%90%E8%A2%AB%E5%AE%B6%E6%9A%B4-%E5%AF%84%E9%A4%8A%E7%88%B6%E6%AF%8D%E6%85%88%E6%84%9B%E6%88%90%E6%A5%B7%E6%A8%A1-073

https://tw.news.yahoo.com/%E7%BF%B9%E8%87%80%E5%B0%B1%E5%BE%97%E7%B7%B4%E6%B7%B1%E8%B9%B2-%E6%8F%9B%E5%81%9A-%E6%A9%8B%E5%BC%8F-%E8%BC%95%E9%AC%86%E4%B8%8A%E6%89%8B-003209994.html
https://tw.news.yahoo.com/%E5%A4%A7%E4%BF%9D%E9%A4%8A%E8%A6%81%E5%81%9A%E5%A5%BD-%E6%89%8D%E8%83%BD%E6%9C%89%E6%95%88%E6%8A%97%E8%A1%B0%E8%80%81-081124112.html
https://tw.news.yahoo.com/%E7%AB%B9%E5%B8%82%E7%99%8C%E7%97%87%E7%AF%A9%E6%AA%A2%E9%96%8B%E8%B7%91-%E9%A1%A7%E5%81%A5%E5%BA%B7%E6%8A%BD%E8%90%AC%E9%87%91-215007079.html
https://tw.news.yahoo.com/22%E5%91%A8%E6%97%A9%E7%94%A2%E5%85%92-%E5%B7%B4%E6%8E%8C%E4%BB%99%E5%AD%90%E6%AD%A1%E5%96%9C%E5%87%BA%E9%99%A2-215007954.html
https://tw.news.yahoo.com/%E9%81%BA%E5%82%B3%E6%80%A7%E9%BB%83%E6%96%91%E9%83%A8%E7%97%85%E8%AE%8A-%E6%89%BE%E5%88%B0%E8%A7%A3%E6%96%B9-160000107.html
https://tw.news.yahoo.com/%E6%83%B3%E6%B8%9B%E8%82%A5-%E9%80%999%E7%A8%AE%E9%AB%98%E8%9B%8B%E7%99%BD%E9%A3%9F%E7%89%A9%E4%B8%8D%E5%8F%AF%E5%B0%91-010000327.html
https://tw.news.yahoo.com/

AttributeError: 'NoneType' object has no attribute 'group'

## Use selenium

In [15]:
from selenium import webdriver
driver = webdriver.Chrome()

In [17]:
cat = categories[0]
driver.get('https://tw.news.yahoo.com/{}/archive'.format(cat))
cat

'politics'

In [18]:
urls = set()
for article in driver.find_elements_by_css_selector('.Cf a'):
    if article.text:
#         urls.add(article.text)
        urls.add(article.get_attribute('href'))
print(len(urls))

28


In [19]:
# used for: https://tw.news.yahoo.com/politics
for article in driver.find_elements_by_css_selector('div#mrt-node-Col1-1-Hero a'):
    if article.text:
#         urls.add(article.text)
        urls.add(article.get_attribute('href'))
print(len(urls))

28


In [204]:
import time
def get_urls(cat):
    driver.get('https://tw.news.yahoo.com/{}'.format(cat))

    urls = set()
    for article in driver.find_elements_by_css_selector('.js-stream-content > div:not(.controller) a:not(.comment-btn-link)'):
        if article.text:
            urls.add(article.get_attribute('href'))
    for article in driver.find_elements_by_css_selector('div#mrt-node-Col1-1-Hero a'):
        if article.text:
            urls.add(article.get_attribute('href'))
    print(len(urls))
    return urls

In [189]:
len(links['politics'])

60